## Context manager

A context manager is an object that responds to a `with` statement. It may returns something. The basic idea is that some action is performed when entering a context and again when exiting it.

```
with context as var:
    # do something
```
translates to
```
# execute context entering code
var = return_from_context_entering_code
# do something
# execute context leaving code
```

The great advantage here is that the "leaving code" is automatically executed whenever we step out of the context!

This proved to be incredibly useful when operations have cleanup code that we need to execute yet that is tedious to write manually and can be forgotten.

### Using `yield`

One way to create a context manager is to have a function that has a `yield`.

_What is `yield`?_: It's like a return, except that the execution stops at the `yield`, lets other code execute and, at some point, **continues** again where the yield was. Examples are:
- iterator: a function that yields elements. Everytime it is called, it is supposed to yield an element and then continue from there
- asynchronous programing: it stops and waits until something else is finished
- in the context manager, as we will see

In [ ]:
import contextlib


@contextlib.contextmanager
def printer(x):
    print(f'we just entered the context manager and will yield {x}')
    yield x
    print(f'Finishing the context manager, exiting')

In [ ]:
with printer(5) as number:
    print(f"we're inside, with number={number}")
print("left manager")

#### Where is this useful

Basically with stateful objects. This includes anything that can be set and changed (mutable objects).

In [ ]:
with open('tmp.txt', 'w') as textfile:
    textfile.write('asdf')

The implementation roughly looks like this:

In [ ]:
import contextlib


@contextlib.contextmanager
def myopen(f, mode):
    opened = open(f, mode)
    yield opened
    opened.close()

**Exercise**: create a context manager that _temporarily_ sets a `'value'` key to 42 of a dict and switches it back to the old value on exit

In [ ]:
testdict = {'value': 11, 'name': 'the answer'}

to be invoked like this

```python
with manager(testdict) as obj:
    # here the value is 42
# here the value is 11
```

In [ ]:
# SOLUTION
@contextlib.contextmanager
def func(x):
    yield x


with func(5) as var1:
    print('inside')
print(var1)

In [ ]:
@contextlib.contextmanager
def set_answer(obj):
    old_value = obj.get('value')
    obj['value'] = 42
    yield obj
    obj['value'] = old_value

## Using a class

Instead of using the `yield`, we can have advanced control over the enter and exit methods by creating a class and implementing the two methods `__enter__` and `__exit__`

In [ ]:
class MyContext:

    def __init__(self, x):
        self.x = x

    def __enter__(self):
        x = self.x
        print('entered')
        return x ** 2

    def __exit__(self, type_, value, traceback):  # but let's not go into things in detail here
        self.x = 42
        print('exited')

In [ ]:
with MyContext(5) as x:
    print(x)

While a class is way more powerful and offers ways to catch exceptions and more in the exit, ususally the functional way is enough and should then be preferred. If it doesn't give you enough flexibility, remember the class, look it up and figure out all the things needed.